In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from data_pipeline import EXPORTED as data_pipeline
from tensorflow import keras

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Total samples : 1139
Total samples : 12000
The total number of ingredients : 24
The lowest number of images per category : 508
The highest number of images per category : 550
The average number of images per category : 547
The total number of ingredients : 501
The average number of ingredients for each dish (recipes5k + food101) : 10


# Loading builders for datasets

In [ ]:
RECIPES5K_DIR = "./drive/MyDrive/FoodCaloModel/Food Datasets/final-dataset/tfrecord/recipes5k/1.0.0"
FOOD101_DIR = "./drive/MyDrive/FoodCaloModel/Food Datasets/final-dataset/tfrecord/food101/1.1.0"


food101_builder = tfds.builder_from_directories([FOOD101_DIR])

In [ ]:
food101_builder.info.splits["train"].num_examples

12000

In [ ]:
def encode_category_ingredients(category_tensor, ingredients_tensor):
    category = str(category_tensor.numpy(), "utf-8")
    ingredients = str(ingredients_tensor.numpy(), "utf-8")
    one_hot_category_tensor = (
        data_pipeline.one_hot_encoder.get_category_one_hot_encoding(category)
    )
    one_hot_ingredients_tensor = (
        data_pipeline.one_hot_encoder.get_ingredients_one_hot_encoding(
            ingredients.split(",")
        )
    )
    return (
        tf.constant(one_hot_category_tensor, dtype=tf.uint8),
        tf.constant(one_hot_ingredients_tensor, dtype=tf.uint8),
    )

In [ ]:
def parse_function(x):
    encoded = tf.py_function(
        encode_category_ingredients,
        [x["category"], x["ingredients"]],
        [tf.uint8, tf.uint8],
    )
    return x["image_raw"], {
        "category_output": encoded[0],
        "calorie_output": x["calorie"],
        "carbs_output": x["carbs"],
        "protein_output": x["protein"],
        "fat_output": x["fat"],
        "ingredients_output": encoded[1],
    }

## Creating dataset for training category, ingredients and nutrients

In [ ]:
# FULL DATASET
(
    food101_train,
    food101_validation,
) = food101_builder.as_dataset(split=["train[:80%]", "train[80%:]"])


train_dataset = food101_train
validation_dataset = food101_validation

In [ ]:
print(f"Total training size : {train_dataset.cardinality().numpy()}")
print(f"Total validation size : {validation_dataset.cardinality().numpy()}")

Total training size : 9600
Total validation size : 2400


In [ ]:
BATCH_SIZE = 256
train_dataset = (
    train_dataset.map(parse_function).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
)
validation_dataset = (
    validation_dataset.map(parse_function).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
)

# Model Development

In [ ]:
class BaseModel:
    def __init__(
        self,
        input_shape,
        total_food_category,
        total_ingredients_category,
        name,
        model_config_name,
    ):
        self.input_shape = input_shape
        self.total_food_category = total_food_category
        self.total_ingredients_category = total_ingredients_category
        self.model = None
        self.name = name
        self.model_config_name = model_config_name

    def build(
        self,
        independent_category_units,
        independent_ingredients_units,
        independent_protein_units,
        independent_calorie_units,
        independent_carbs_units,
        independent_fat_units,
        shared_units,
    ):
        raise NotImplementedError

    def compile_model(self, optimizer):
        assert self.model is not None
        category_classification_loss = keras.losses.CategoricalCrossentropy()
        calorie_regression_loss = keras.losses.MeanAbsoluteError()
        carbs_regression_loss = keras.losses.MeanAbsoluteError()
        protein_regression_loss = keras.losses.MeanAbsoluteError()
        fat_regression_loss = keras.losses.MeanAbsoluteError()
        ingredient_multilabel_loss = keras.losses.BinaryCrossentropy()
        category_classification_metrics = (
            [
                keras.metrics.CategoricalAccuracy(name="acc"),
                keras.metrics.Precision(name="precision"),
                keras.metrics.Recall(name="recall"),
            ],
        )

        calorie_regression_metrics = [keras.metrics.MeanAbsoluteError(name="MAE")]
        carbs_regression_metrics = [keras.metrics.MeanAbsoluteError(name="MAE")]
        protein_regression_metrics = [keras.metrics.MeanAbsoluteError(name="MAE")]
        fat_regression_metrics = [keras.metrics.MeanAbsoluteError(name="MAE")]
        ingredient_multilabel_metrics = (
            [
                keras.metrics.Precision(name="precision"),
                keras.metrics.Recall(name="recall"),
            ],
        )
        category_classification_loss_weights = 1.0
        ingredient_multilabel_loss_weights = 1.0
        calorie_regression_loss_weights = 1.0
        carbs_regression_loss_weights = 1.0
        protein_regression_loss_weights = 1.0
        fat_regression_loss_weights = 1.0

        self.model.compile(
            optimizer=optimizer,
            loss={
                "category_output": category_classification_loss,
                "calorie_output": calorie_regression_loss,
                "carbs_output": carbs_regression_loss,
                "protein_output": protein_regression_loss,
                "fat_output": fat_regression_loss,
                "ingredients_output": ingredient_multilabel_loss,
            },
            metrics={
                "category_output": category_classification_metrics,
                "calorie_output": calorie_regression_metrics,
                "carbs_output": carbs_regression_metrics,
                "protein_output": protein_regression_metrics,
                "fat_output": fat_regression_metrics,
                "ingredients_output": ingredient_multilabel_metrics,
            },
            loss_weights={
                "category_output": category_classification_loss_weights,
                "calorie_output": calorie_regression_loss_weights,
                "carbs_output": carbs_regression_loss_weights,
                "protein_output": protein_regression_loss_weights,
                "fat_output": fat_regression_loss_weights,
                "ingredients_output": ingredient_multilabel_loss_weights,
            },
        )

    def freeze_category_classification_layers(self):
        assert self.model is not None
        submodel = self.model.get_layer("category_output")
        submodel.trainable = False

    def freeze_convolution_base(self):
        assert self.model is not None
        submodel = self.model.get_layer("efficientnetB1")
        submodel.trainable = False

    def get_callbacks(self):
        tensorboard_dir = f"./drive/MyDrive/FoodCaloModel/models/logs/{self.name}/{self.model_config_name}"
        checkpoint_dir = f"./drive/MyDrive/FoodCaloModel/temp/checkpoint/{self.name}/{self.model_config_name}"
        checkpoint_model_path = checkpoint_dir + "/model"
        checkpoint_weights_path = checkpoint_dir + "/weights/ckpt"
        checkpoint_model_callback = keras.callbacks.ModelCheckpoint(
            checkpoint_model_path, monitor="val_loss", save_best_only=True, mode="min"
        )
        checkpoint_weights_callback = keras.callbacks.ModelCheckpoint(
            checkpoint_weights_path,
            monitor="val_loss",
            save_best_only=True,
            save_weights_only=True,
            mode="min",
        )
        reduce_lr_callback = keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss", factor=0.5, patience=2, min_lr=1e-6, verbose=1
        )
        tensorboard_callback = keras.callbacks.TensorBoard(log_dir=tensorboard_dir)
        early_stopping_callback = keras.callbacks.EarlyStopping("val_loss", patience=8)
        return [
            tensorboard_callback,
            early_stopping_callback,
            checkpoint_model_callback,
            checkpoint_weights_callback,
            reduce_lr_callback,
        ]

    # Layers from bottom to top (classifier)  #
    def get_input_layer(self):
        return keras.Input(shape=self.input_shape)

    def get_augmentation_layers(self):
        input_layer = keras.layers.Input(shape=self.input_shape)
        augmentation_layer = keras.layers.RandomFlip()(input_layer)
        augmentation_layer = keras.layers.RandomRotation(0.2)(augmentation_layer)
        augmentation_layer = keras.layers.RandomZoom(0.2)(augmentation_layer)
        augmentation_layer = keras.layers.RandomTranslation(0.2, 0.2)(augmentation_layer)
        augmentation_layer = keras.layers.RandomRotation(0.2)(augmentation_layer)

        return keras.Model(
            inputs=input_layer, outputs=augmentation_layer, name="augmentation_layers"
        )

    def get_preprocess_layers(self, input_tensor):
        raise NotImplementedError

    def get_convolution_block(self):
        raise NotImplementedError

    def get_shared_layers(self, input_tensor, *num_units):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        shared_layer = keras.layers.Flatten()(input_layer)
        shared_layer = keras.layers.Dense(
            num_units[0], activation="relu", name="shared_dense_1"
        )(shared_layer)
        shared_layer = keras.layers.BatchNormalization()(shared_layer)
        shared_layer = keras.layers.Dropout(0.2)(shared_layer)
        shared_layer = keras.layers.BatchNormalization()(shared_layer)
        output_layer = keras.layers.Dropout(0.2)(shared_layer)
        return keras.Model(
            inputs=input_layer, outputs=output_layer, name="shared_layers"
        )

    def get_category_classification_layers(
        self, input_tensor, total_categories, *num_units
    ):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        x = keras.layers.Dense(
            num_units[0], activation="relu", name="category_dense_1"
        )(input_layer)
        x = keras.layers.BatchNormalization()(x)
        category_classification_layer = keras.layers.Dense(
            total_categories, activation="softmax", name="category_output_layer"
        )(x)
        output_model = keras.Model(
            inputs=input_layer,
            outputs=category_classification_layer,
            name="category_output",
        )
        return output_model

    def get_calorie_regression_layers(self, input_tensor, *num_units):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        x = keras.layers.Dense(num_units[0], activation="relu", name="calorie_dense_1")(
            input_layer
        )
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Dense(num_units[1], activation="relu", name="calorie_dense_2")(
            x
        )
        x = keras.layers.BatchNormalization()(x)
        calorie_regression_layers = keras.layers.Dense(1, name="calorie_output_layer")(
            x
        )
        return keras.Model(
            inputs=input_layer, outputs=calorie_regression_layers, name="calorie_output"
        )

    def get_carbs_regression_layers(self, input_tensor, *num_units):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        x = keras.layers.Dense(num_units[0], activation="relu", name="carbs_dense_1")(
            input_layer
        )
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Dense(num_units[1], activation="relu", name="carbs_dense_2")(x)
        x = keras.layers.BatchNormalization()(x)
        carbs_regression_layers = keras.layers.Dense(1, name="carbs_output_layer")(x)
        return keras.Model(
            inputs=input_layer, outputs=carbs_regression_layers, name="carbs_output"
        )

    def get_protein_regression_layers(self, input_tensor, *num_units):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        x = keras.layers.Dense(num_units[0], activation="relu", name="protein_dense_1")(
            input_layer
        )
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Dense(num_units[1], activation="relu", name="protein_dense_2")(
            x
        )
        x = keras.layers.BatchNormalization()(x)
        protein_regression_layers = keras.layers.Dense(1, name="protein_output_layer")(
            x
        )
        return keras.Model(
            inputs=input_layer, outputs=protein_regression_layers, name="protein_output"
        )

    def get_fat_regression_layers(self, input_tensor, *num_units):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        x = keras.layers.Dense(num_units[0], activation="relu", name="fat_dense_1")(
            input_layer
        )
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Dense(num_units[1], activation="relu", name="fat_dense_2")(x)
        x = keras.layers.BatchNormalization()(x)
        fat_regression_layers = keras.layers.Dense(1, name="fat_output_layer")(x)
        return keras.Model(
            inputs=input_layer, outputs=fat_regression_layers, name="fat_output"
        )

    def get_ingredients_multilabel_layers(
        self, input_tensor, total_ingredients, *num_units
    ):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        ingredients_multilabel_layers = keras.layers.Dense(
            num_units[0], activation="relu", name="ingredients_dense_1"
        )(input_layer)
        ingredients_multilabel_layers = keras.layers.BatchNormalization()(
            ingredients_multilabel_layers
        )
        ingredients_multilabel_layers = keras.layers.Dense(
            num_units[1], activation="relu", name="ingredients_dense_2"
        )(ingredients_multilabel_layers)
        ingredients_multilabel_layers = keras.layers.BatchNormalization()(
            ingredients_multilabel_layers
        )
        ingredients_multilabel_layers = keras.layers.Dense(
            num_units[2], activation="relu", name="ingredients_dense_3"
        )(ingredients_multilabel_layers)
        ingredients_multilabel_layers = keras.layers.BatchNormalization()(
            ingredients_multilabel_layers
        )
        output_layers = keras.layers.Dense(
            total_ingredients, activation="sigmoid", name="ingredients_output_layer"
        )(ingredients_multilabel_layers)
        return keras.Model(
            inputs=input_layer, outputs=output_layers, name="ingredients_output"
        )

    # End of layers  #

    def load_model(self, path):
        self.model = keras.models.load_model(path)

    def print_summary(self):
        assert (
            self.model is not None
        ), "Please run build_and_compile before printing summary."
        self.model.summary(expand_nested=True, show_trainable=True)

    def save_model(self, path):
        assert self.model is not None
        self.model.save(path, save_format="h5")

    def train_model(self, **kwargs):
        assert self.model is not None, "No model found."
        return self.model.fit(**kwargs, callbacks=self.get_callbacks())

    def unfreeze_category_classification_layers(self):
        assert self.model is not None
        submodel = self.model.get_layer("category_output")
        submodel.trainable = False

    def unfreeze_convolution_base(self, fine_tune_at=0):
        assert self.model is not None
        submodel = self.model.get_layer("efficientnetb1")
        submodel.trainable = True
        for layer in submodel.layers[:fine_tune_at]:
            layer.trainable = False

## Flat Model

In [ ]:
class FlatModel(BaseModel):
    def __init__(
        self,
        input_shape,
        total_food_category,
        total_ingredients_category,
        name,
        model_config_name,
    ):
        super().__init__(
            input_shape,
            total_food_category,
            total_ingredients_category,
            name,
            model_config_name,
        )

    def build(
        self,
        shared_units,
        independent_category_units,
        independent_protein_units,
        independent_fat_units,
        independent_carbs_units,
        independent_calorie_units,
        independent_ingredients_units,
    ):
        model_inputs = self.get_input_layer()
        prev_layer = self.get_augmentation_layers()(model_inputs)
        prev_layer = self.get_preprocess_layers(prev_layer)(prev_layer)
        prev_layer = self.get_convolution_block()(prev_layer)
        prev_layer = self.get_shared_layers(prev_layer, *shared_units)(prev_layer)
        category_classification_head = self.get_category_classification_layers(
            prev_layer, self.total_food_category, *independent_category_units
        )(prev_layer)
        ingredients_multilabel_head = self.get_ingredients_multilabel_layers(
            prev_layer, self.total_ingredients_category, *independent_ingredients_units
        )(prev_layer)
        calorie_regression_head = self.get_calorie_regression_layers(
            prev_layer, *independent_calorie_units
        )(prev_layer)
        carbs_regression_head = self.get_carbs_regression_layers(
            prev_layer, *independent_carbs_units
        )(prev_layer)
        protein_regression_head = self.get_protein_regression_layers(
            prev_layer, *independent_protein_units
        )(prev_layer)
        fat_regression_head = self.get_fat_regression_layers(
            prev_layer, *independent_fat_units
        )(prev_layer)

        model = keras.Model(
            inputs=model_inputs,
            outputs=[
                category_classification_head,
                ingredients_multilabel_head,
                calorie_regression_head,
                carbs_regression_head,
                protein_regression_head,
                fat_regression_head,
            ],
            name=self.name,
        )
        self.model = model

### EfficientNetB1 (Best)

In [ ]:
class FlatEfficientNetB1Model(FlatModel):
    def __init__(
        self,
        input_shape,
        total_food_category,
        total_ingredients_category,
        model_config_name,
    ):
        super().__init__(
            input_shape,
            total_food_category,
            total_ingredients_category,
            "flat_efficientnetB1",
            model_config_name,
        )

    def get_preprocess_layers(self, input_tensor):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        output_layer = keras.applications.efficientnet.preprocess_input(input_layer)
        return keras.Model(
            inputs=input_layer, outputs=output_layer, name="preprocessing_layers"
        )

    def get_convolution_block(self):
        efficientnet_convolution_layers = (
            keras.applications.efficientnet.EfficientNetB1(
                input_shape=self.input_shape,
                include_top=False,
                weights="imagenet",
                pooling="avg",
            )
        )
        efficientnet_convolution_layers.trainable = False
        return efficientnet_convolution_layers

#### Train on main dataset (recipes5k + food101) with weights transfer from ingredients dataset

In [ ]:
flat_efficientnet = FlatEfficientNetB1Model(
    input_shape=(224, 224, 3),
    total_food_category=len(data_pipeline.one_hot_encoder.all_food_categories),
    total_ingredients_category=len(data_pipeline.one_hot_encoder.all_ingredients),
    model_config_name="finalIngredients",
)

flat_efficientnet.build(
    shared_units=[2048],
    independent_category_units=[512],
    independent_ingredients_units=[1024, 512, 256],
    independent_protein_units=[64, 32],
    independent_fat_units=[64, 32],
    independent_carbs_units=[64, 32],
    independent_calorie_units=[64, 32],
)
flat_efficientnet.freeze_category_classification_layers()
flat_efficientnet.compile_model(keras.optimizers.Adam())

flat_efficientnet.print_summary()

Model: "flat_efficientnetB1"
_____________________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  Trainable  
 input_6 (InputLayer)        [(None, 224, 224, 3)]        0         []                            Y          
                                                                                                             
 augmentation_layers (Funct  (None, 224, 224, 3)          0         ['input_6[0][0]']             Y          
 ional)                                                                                                      
|¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯|
| input_7 (InputLayer)       [(None, 224, 224, 3)]        0         []                            Y         |
|                                                                                          

In [ ]:
pip install --upgrade tensorflow

In [ ]:
flat_efficientnet.train_model(
    x=train_dataset,
    validation_data=validation_dataset,
    epochs=50,
    verbose=2,
)

Epoch 1/50
38/38 - 2112s - loss: 6.0573 - category_output_loss: 2.1692 - ingredients_output_loss: 0.6852 - calorie_output_loss: 1.7315 - carbs_output_loss: 0.4935 - protein_output_loss: 0.4545 - fat_output_loss: 0.5234 - category_output_acc: 0.3915 - category_output_precision: 0.6747 - category_output_recall: 0.2355 - ingredients_output_precision: 0.0360 - ingredients_output_recall: 0.6388 - calorie_output_MAE: 1.7315 - carbs_output_MAE: 0.4935 - protein_output_MAE: 0.4545 - fat_output_MAE: 0.5234 - val_loss: 4.4674 - val_category_output_loss: 1.7767 - val_ingredients_output_loss: 0.6373 - val_calorie_output_loss: 1.5203 - val_carbs_output_loss: 0.2302 - val_protein_output_loss: 0.1580 - val_fat_output_loss: 0.1450 - val_category_output_acc: 0.6421 - val_category_output_precision: 0.9405 - val_category_output_recall: 0.0329 - val_ingredients_output_precision: 0.0623 - val_ingredients_output_recall: 0.6593 - val_calorie_output_MAE: 1.5203 - val_carbs_output_MAE: 0.2302 - val_protein_out

KeyboardInterrupt: ignored